In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'SNP'

import sys

import pandas as pd
from from_root import from_root
from ib_insync import IB, LimitOrder, util
from loguru import logger

from src.utils import Vars, get_pickle, place_orders, quick_pf

util.startLoop()

# Set the root
ROOT = from_root()
_vars = Vars(MARKET)
PORT = port = _vars.PORT
DATAPATH = ROOT / 'data' / MARKET

# set vscode path
sys.path.append(str(ROOT / 'src'))

# To import use 
# from src.<filename> import <function>

In [2]:
df_tgt = get_pickle(DATAPATH / "df_naked_targets.pkl")
df = df_tgt.dropna(subset=['expPrice'])

OPT_COLS = _vars.OPT_COLS[0]

In [3]:
# REMOVE PORTFOLIO SYMBOLS FROM TARGET
# ====================================

# Get the portfolio
with IB().connect(port=PORT) as ib:
    df_pf = quick_pf(ib)

# Remove targets which are already in the portfolio
if isinstance(df_pf, pd.DataFrame):
    df = df[~df.symbol.isin(set(df_pf.symbol))]

In [4]:
# REMOVE OPEN ORDER SYMBOLS
# =========================

with IB().connect(port=PORT) as ib:
    ib.reqAllOpenOrders()  # To kickstart collection of open orders
    # ib.sleep(0.3)
    trades = ib.trades()

if trades:
    all_trades_df = (util.df(t.contract for t in trades).join(
        util.df(t.orderStatus
                for t in trades)).join(util.df(t.order for t in trades),
                                        lsuffix="_"))

    all_trades_df.rename({"lastTradeDateOrContractMonth": "expiry"},
                            axis="columns",
                            inplace=True)
    trades_cols = ["conId", "symbol", "secType", "expiry",
        "strike", "right", "orderId", "permId", "action",
        "totalQuantity", "lmtPrice", "status",]

    dfo = all_trades_df[trades_cols]
    df_openords = dfo[all_trades_df.status.isin(_vars. ACTIVE_STATUS)]

else:
    df_openords = None
    

try:
    if df_openords:  # There are some open orders
        df = df[~df.symbol.isin(set(df_openords.symbol))]
except ValueError:
    pass

In [5]:
# Make (contract, order) tuple
cos = [(contract , LimitOrder('Sell', qty, price))
    for contract, qty, price in zip(df.contract, df.lot_size, df.expPrice)]

In [6]:
df[OPT_COLS]

,conId,symbol,undPrice,expiry,strike,right,dte,sdev,optPrice,expPrice,margin,comm,lot_size
0,672065704,MMM,106.06,20240216,130.0,C,28.721993,3.031654,NaN,29.6,445.66,1.549665,1.0
1,676178775,MMM,106.06,20240223,130.0,C,35.721993,2.718436,NaN,36.6,447.33,1.549698,1.0
2,677398653,MMM,106.06,20240301,130.0,C,42.721993,2.485772,NaN,42.7,439.00,1.549682,1.0
3,677039003,ABT,113.82,20240209,128.0,C,21.721834,2.538780,NaN,40.1,810.15,1.549700,1.0
4,636817600,ABT,113.82,20240216,130.0,C,28.721834,2.519240,NaN,47.9,736.37,1.549666,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,678820752,XSP,475.55,20240206,446.0,P,18.719161,-2.317779,0.38,0.4,NaN,NaN,1.0
1301,678820885,XSP,475.55,20240207,446.0,P,19.719161,-2.258245,0.42,0.4,NaN,NaN,1.0
1302,675249976,XSP,475.55,20240208,445.0,P,20.719161,-2.277629,0.45,0.4,NaN,NaN,1.0
1303,651807598,XSP,475.55,20240301,435.0,P,42.719161,-2.105413,0.93,0.9,NaN,NaN,1.0


In [7]:
# PLACE ORDERS
# +++++++++++++
with IB().connect(port=PORT) as ib:

    orders_placed = place_orders(ib, cos)

    logger.info(f"Successfully placed {len(orders_placed)} orders!")

100%|██████████| 50/50 [00:38<00:00,  1.29it/s]
2024-01-18 23:13:24.855 | INFO     | __main__:<module>:7 - Successfully placed 1246 orders!
